In [1]:
import requests
!pip install folium
import folium
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

     |████████████████████████████████| 102kB 10.8MB/s ta 0:00:01


In [2]:
input_html ='https://frenchmoments.eu/arrondissements-of-paris'
html_data = requests.get(input_html).content

parsed_data = BeautifulSoup(html_data,'html.parser')

In [7]:
borough_list = []
for idx in parsed_data.select('h4'):
    if idx.text.find('arrondissement') > -1:
       start_pos = idx.text.find('arrondissement') + len('arrondissement') + 2
       end_pos = idx.text.find('arrondissement')  + len('arrondissement')
       borough_list.append((idx.text[0:end_pos]))
    
neighborhood_list = []
parsed_data = BeautifulSoup(html_data,'html.parser')
for idx in parsed_data.select('h4'):
    if idx.text.find('arrondissement') > -1:
       start_pos = idx.text.find('arrondissement') + len('arrondissement') + 2
       end_pos = len(idx.text)-1
       neighborhood_list.append(idx.text[start_pos:end_pos])

print("Borough list - ", borough_list)
print("Neighborhood list - ", neighborhood_list)

Borough list -  ['1st arrondissement', '2nd arrondissement', '3rd arrondissement', '4th arrondissement', '5th arrondissement', '6th arrondissement', '7th arrondissement', '8th arrondissement', '9th arrondissement', '10th arrondissement', '11th arrondissement', '12th arrondissement', '13th arrondissement', '14th arrondissement', '15th arrondissement', '16th arrondissement', '17th arrondissement', '18th arrondissement', '19th arrondissement', '20th arrondissement']
Neighborhood list -  ['Louvre', 'Bourse', 'Marais – north', 'Marais – south, Notre-Dame', 'Latin Quarter', 'St. Germain-des-Prés, Luxembourg Garden', 'Eiffel Tower, Invalides', 'Arc de Triomphe, Champs-Élysées', 'Opéra Garnier, Grands Boulevards', 'Canal Saint-Martin', 'Faubourg St. Antoine – Oberkampf', 'Bercy, Bois de Vincennes', 'Tolbiac, Gobelins, Paris Rive Gauche', 'Montparnasse', 'Vaugirard, Grenelle', 'Passy, Auteuil', 'Batignolles, Monceau', 'Montmartre', 'La Villette, Buttes-Chaumont', 'Belleville, Ménilmontant']


In [3]:
postal_code = []
for idx in parsed_data.select('span'):
    if (idx.text.find('Postal Code') > -1):
       postal_code.append(idx.text[-5:])

lat_list = []
long_list = []
x_nom = Nominatim(user_agent='Foursquare')
for idx, idy in zip(borough_list, postal_code):
    x_nom_out = x_nom.geocode('{},{}'.format(idx,idy)).raw
    lat_list.append(x_nom_out['lat'])
    long_list.append(x_nom_out['lon'])

In [5]:
df_paris = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Postal Code', 'Latitude','Longitude'])
df_paris['Borough'] = borough_list
df_paris['Neighborhood'] = neighborhood_list
df_paris['Postal Code'] = postal_code
df_paris['Latitude'] = lat_list
df_paris['Longitude'] = long_list
df_paris

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,1st arrondissement,Louvre,75001,48.8646144,2.334396
1,2nd arrondissement,Bourse,75002,48.868743,2.341688
2,3rd arrondissement,Marais – north,75003,48.864212,2.360936
3,4th arrondissement,"Marais – south, Notre-Dame",75004,48.8562021,2.3556193
4,5th arrondissement,Latin Quarter,75005,48.8460591,2.3445228
5,6th arrondissement,"St. Germain-des-Prés, Luxembourg Garden",75006,48.8504333,2.3329507
6,7th arrondissement,"Eiffel Tower, Invalides",75007,48.8570281,2.3201953
7,8th arrondissement,"Arc de Triomphe, Champs-Élysées",75008,48.87225991745448,2.311689291751072
8,9th arrondissement,"Opéra Garnier, Grands Boulevards",75009,48.876019,2.339962
9,10th arrondissement,Canal Saint-Martin,75010,48.876106,2.35991


In [6]:
print("Total number of arrondissement in Paris: ", df_paris.shape[0])

Total number of arrondissement in Paris:  20


In [9]:
x_nom = Nominatim(user_agent='Foursquare')
x_nom_out = x_nom.geocode('Paris,france')
paris_lat = x_nom_out.latitude
paris_long = x_nom_out.longitude

In [10]:
paris_map_1 = folium.Map([paris_lat,paris_long],zoom_start =13)

for lat, long, borough,neigh in zip(df_paris['Latitude'],df_paris['Longitude'],df_paris['Borough'],df_paris['Neighborhood']):
    label = folium.Popup('{},{}'.format(borough,neigh))
    folium.Marker(
     [lat,long],
     popup = label,
     color = 'red',
     radius = 5,
     fill = True,
     fill_color = 'blue',
     opacity=0.8).add_to(paris_map_1)
paris_map_1

In [11]:
CLIENT_ID = '0WAUW3LYUG1C2AHQKJBAMTWCLGZPXDRTRBCZ3EZI4EEZKZHL'
CLIENT_SECRET = 'TRBZA1ORTZTYWOBOPIDNEB0HGNT24RGEKYF3IGK2VV2VBM31'
VERSION = '20200601'
RADIUS = 1500

In [12]:
venue_list = []

for lat, long, borough, neigh in zip(df_paris['Latitude'],df_paris['Longitude'],df_paris['Borough'],df_paris['Neighborhood']):
    explore_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,RADIUS,'toys')
    explore_out = requests.get(explore_url).json()
    for idx in explore_out['response']['venues']:
        venue_name = idx['name']
        try:
           venue_type = idx['categories'][0]['name']
        except:
           venue_type = ''
        venue_lat = idx['location']['lat']
        venue_long = idx['location']['lng']
        venue_list.append([borough,neigh,lat,long,venue_name,venue_type,venue_lat,venue_long])

In [27]:
df_paris_venue = pd.DataFrame(venue for venue in venue_list)
df_paris_venue.columns = ['Borough','Neighbourhood','Borough Latitude','Borough Longitude','Venue Name','Venue Type','Venue Latitude','Venue Longitude']
df_paris_venue = df_paris_venue[df_paris_venue['Venue Type'].str.contains('Toy')]
df_paris_venue

,Borough,Neighbourhood,Borough Latitude,Borough Longitude,Venue Name,Venue Type,Venue Latitude,Venue Longitude
0,1st arrondissement,Louvre,48.8646144,2.334396,"Toys""R""Us",Toy / Game Store,48.876202,2.324837
2,1st arrondissement,Louvre,48.8646144,2.334396,Pulp's Toys,Toy / Game Store,48.851367,2.346638
4,2nd arrondissement,Bourse,48.868743,2.341688,"Toys""R""Us",Toy / Game Store,48.876202,2.324837
7,2nd arrondissement,Bourse,48.868743,2.341688,Pulp's Toys,Toy / Game Store,48.851367,2.346638
9,3rd arrondissement,Marais – north,48.864212,2.360936,"Toys""R""Us",Toy / Game Store,48.859749,2.369441
11,3rd arrondissement,Marais – north,48.864212,2.360936,Manga Toys,Toy / Game Store,48.854262,2.376240
12,3rd arrondissement,Marais – north,48.864212,2.360936,Pulp's Toys,Toy / Game Store,48.851367,2.346638
14,4th arrondissement,"Marais – south, Notre-Dame",48.8562021,2.3556193,"Toys""R""Us",Toy / Game Store,48.859749,2.369441
15,4th arrondissement,"Marais – south, Notre-Dame",48.8562021,2.3556193,Pulp's Toys,Toy / Game Store,48.851367,2.346638
18,4th arrondissement,"Marais – south, Notre-Dame",48.8562021,2.3556193,Manga Toys,Toy / Game Store,48.854262,2.376240


In [15]:
paris_map_2 = folium.Map([paris_lat,paris_long],zoom_start=12.5)
for lat, long, venue in zip(df_paris_venue['Venue Latitude'], df_paris_venue['Venue Longitude'], df_paris_venue['Venue Name']):
    folium.CircleMarker(
    [lat,long],
    color = 'red',
    radius=3,
    fill = True,
    fill_color = 'Blue',
    fill_opacity=0.6,
    popup=folium.Popup(venue)
    ).add_to(paris_map_2)
paris_map_2

In [16]:
df_paris_venue.groupby(['Borough']).count()

,Neighbourhood,Borough Latitude,Borough Longitude,Venue Name,Venue Type,Venue Latitude,Venue Longitude
Borough,,,,,,,
10th arrondissement,1,1,1,1,1,1,1
11th arrondissement,1,1,1,1,1,1,1
14th arrondissement,1,1,1,1,1,1,1
16th arrondissement,1,1,1,1,1,1,1
17th arrondissement,1,1,1,1,1,1,1
18th arrondissement,1,1,1,1,1,1,1
1st arrondissement,6,6,6,6,6,6,6
2nd arrondissement,5,5,5,5,5,5,5
3rd arrondissement,4,4,4,4,4,4,4


In [166]:
df_temp = pd.get_dummies(df_paris_venue['Venue Type'])
df_temp.insert(0,'Borough',df_paris_venue['Borough'])
df_temp = df_temp.groupby(['Borough'],as_index=False).sum()

top_venues = 15
col_list = ['st','nd','rd']
columns_list = ['Borough']
for idx in range(top_venues):
    if idx < 3:
        columns_list.append('{}{} most common venue'.format((idx+1),col_list[idx]))
    else:
        columns_list.append('{}th most common venue'.format(idx+1))
                            
df_paris_common_venue = pd.DataFrame(columns=columns_list)
df_paris_common_venue['Borough'] = df_temp['Borough']
for idx in range(df_temp.shape[0]):
    row = df_temp.iloc[idx][1:]
    df_paris_common_venue.iloc[idx][1:] = row.sort_values(ascending=False).head(15).index.values
df_paris_common_venue

,Borough,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,11th most common venue,12th most common venue,13th most common venue,14th most common venue,15th most common venue
0,10th arrondissement,French Restaurant,Bistro,African Restaurant,Mediterranean Restaurant,Coffee Shop,Café,Creperie,Cocktail Bar,Salad Place,Farmers Market,Flower Shop,Food & Drink Shop,Shanxi Restaurant,Garden,Breakfast Spot
1,11th arrondissement,Café,Italian Restaurant,French Restaurant,Cocktail Bar,Pastry Shop,Art Museum,Restaurant,Moroccan Restaurant,Plaza,South American Restaurant,Sandwich Place,Comfort Food Restaurant,Ethiopian Restaurant,Gastropub,Mediterranean Restaurant
2,12th arrondissement,French Restaurant,Bistro,Hotel,Bakery,Ice Cream Shop,Supermarket,Plaza,Creperie,Doner Restaurant,Electronics Store,Farmers Market,Chinese Restaurant,Cheese Shop,Café,Grocery Store
3,13th arrondissement,Vietnamese Restaurant,Hotel,Thai Restaurant,French Restaurant,Convenience Store,Sushi Restaurant,Creperie,Pub,Indian Restaurant,Park,Museum,Movie Theater,Café,Gaming Cafe,Mexican Restaurant
4,14th arrondissement,French Restaurant,Wine Shop,Bar,Vietnamese Restaurant,Italian Restaurant,Bakery,Wine Bar,Pizza Place,Creperie,Southwestern French Restaurant,Brasserie,Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop
5,15th arrondissement,French Restaurant,Italian Restaurant,Bar,Lebanese Restaurant,Indian Restaurant,Gym / Fitness Center,Restaurant,Coffee Shop,Russian Restaurant,Park,Ethiopian Restaurant,Seafood Restaurant,Gastropub,Spanish Restaurant,Korean Restaurant
6,16th arrondissement,Bakery,French Restaurant,Hotel,Garden,Restaurant,Café,Wine Shop,Supermarket,Diner,Pizza Place,Sandwich Place,Chinese Restaurant,Italian Restaurant,Train Station,Asian Restaurant
7,17th arrondissement,Wine Bar,Bar,Bookstore,Italian Restaurant,French Restaurant,Gym / Fitness Center,Indian Restaurant,Creperie,Restaurant,Coffee Shop,Farmers Market,Noodle House,Modern European Restaurant,Steakhouse,Pizza Place
8,18th arrondissement,French Restaurant,Bar,Italian Restaurant,Café,Jazz Club,Breakfast Spot,Japanese Restaurant,Wine Bar,Middle Eastern Restaurant,Gastropub,Pizza Place,Convenience Store,Scenic Lookout,Seafood Restaurant,Cheese Shop
9,19th arrondissement,Bar,Bistro,French Restaurant,Beer Bar,Brewery,Performing Arts Venue,Café,Canal,Seafood Restaurant,Restaurant,Sandwich Place,Cocktail Bar,Coffee Shop,Rock Club,Farmers Market


In [180]:
df_paris_updated = df_paris.join(df_paris_common_venue.set_index('Borough'),on=['Borough'])